Let's downloads Qdrant’s Docker image, then run Qdrant locally, expose its APIs, and ensure stored data survives container restarts.

```bash
docker pull qdrant/qdrant

docker run -p 6333:6333 -p 6334:6334 \
   -v "$(pwd)/qdrant_storage:/qdrant/storage:z" \
   qdrant/qdrant

In [30]:
import requests
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
import os
from qdrant_client import QdrantClient, models

In [31]:
embedding_dim = 512
collection_name = 'zoomcamp-faq'
embedding_model_handle = 'jinaai/jina-embeddings-v2-small-en'
llm_model_handle = 'meta-llama/llama-4-scout-17b-16e-instruct'

In [32]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
docs_raw = docs_response.json()

In [33]:
documents = []
for course in docs_raw:
	course_name = course['course']
	for doc in course['documents']:
		doc['course'] = course_name
		documents.append(doc)

In [34]:
qdrant_client = QdrantClient("http://localhost:6333")

qdrant_client.delete_collection(collection_name=collection_name)
qdrant_client.create_collection(
    collection_name=collection_name, 
    vectors_config=models.VectorParams(
        size=embedding_dim, 
        distance=models.Distance.COSINE
    )
)

True

In [35]:
points = []

for id, doc in enumerate(documents):
    text = f"Question: {doc['question']}\nAnswer: {doc['text']}"
    point = models.PointStruct(
        id=id, 
        vector=models.Document(text=text, model=embedding_model_handle), 
        payload=doc
    )
    points.append(point)

In [36]:
qdrant_client.upsert(
    collection_name=collection_name, 
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [37]:
qdrant_client.create_payload_index(
    collection_name=collection_name, 
    field_name='course', 
    field_schema='keyword' # exact matching on string metadata fields
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [38]:
def vector_search(query, course='machine-learning-zoomcamp', limit=1):
    results = qdrant_client.query_points(
        collection_name=collection_name, 
        query=models.Document(text=query, model=embedding_model_handle),
        query_filter=models.Filter(
            must=[
                models.FieldCondition(
                    key='course',
                    match=models.MatchValue(value=course)
                )
            ]
        ), 
        limit=limit, 
        with_payload=True
    )
    return results

In [39]:
load_dotenv(find_dotenv()) # automatic search
groq_api_key = os.getenv("GROQ_API_KEY")

In [40]:
groq_client = OpenAI(api_key=groq_api_key, base_url='https://api.groq.com/openai/v1')

In [41]:
def build_prompt(query, search_results):	
	# It's a good practice to give the llm you're use a role
	prompt_template = """
You're a teaching assistant. Answer the QUESTION below using the CONTEXT provided below. 
Use only the information in the CONTEXT that's relevant to the QUESTION (ignore irrelavant ones).

QUESTION: {question}

CONTEXT: {context}
"""

	context = ''
	for doc in search_results.points:
		context += f'section: {doc.payload['section']}\nquestion: {doc.payload['question']}\nanswer: {doc.payload['text']}\n\n'

	prompt = prompt_template.format(question=query, context=context).strip()
	return prompt

In [42]:
def llm(prompt):
	response = groq_client.chat.completions.create(
		model=llm_model_handle, 
		messages=[
			{'role': 'user', 'content': prompt}
		]
	)
	return response.choices[0].message.content

In [43]:
def rag(query):
    search_results = vector_search(query, limit=5)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [44]:
query = "the course has already started, can I still enroll?"
print(rag(query))

Yes, you can still enroll in the course even though it has already started. You won't be able to submit some of the homeworks, but you can still take part in the course. To be eligible for a certificate, you need to submit 2 out of 3 course projects and review 3 peers' projects by the deadline.
